## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle

In [2]:
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

objpoints = [] 
imgpoints = [] 

images = glob.glob('../camera_cal/calibration*.jpg')

In [3]:
def cal_undistort(img, idx, objpoints, imgpoints): 
    #store the object points and image points 
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        write_name='../output_images/corners_found'+str(idx)+'.jpg'
        cv2.imwrite(write_name,img)

In [4]:
for idx,fname in enumerate(images):
    img = cv2.imread(fname)
    cal_undistort(img,idx, objpoints, imgpoints)
    
#compute the camera matrix and distortion coefficients
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (img.shape[1],img.shape[0]), None, None)

In [5]:
dist_pickle={}
dist_pickle["mtx"]=mtx
dist_pickle['dist']=dist
pickle.dump(dist_pickle, open('../calibration.pickle.p','wb'))

In [6]:
#Obtain an example of a distortion corrected calibration image.
img=cv2.imread('../camera_cal/calibration2.jpg')
undist = cv2.undistort(img, mtx, dist, None, mtx)
write_name='../output_images/undist2.jpg'
cv2.imwrite(write_name,undist)

True

In [7]:
def abs_sobel_thresh(img, sx_thresh=(15,255), sy_thresh=(25,255)):  
    #Use gradients create a thresholded binary image
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    grad = np.zeros_like(img[:,:,0])
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
    sxbinary = np.zeros_like(scaled_sobelx)
    sybinary = np.zeros_like(scaled_sobely)
  
    sxbinary[(scaled_sobelx >= sx_thresh[0]) & (scaled_sobelx <= sx_thresh[1])] = 1
    sybinary[(scaled_sobely >= sy_thresh[0]) & (scaled_sobely <= sy_thresh[1])] = 1
    
    grad[(sxbinary==1) & (sybinary==1)]=255
    return grad

In [8]:
def color_threshold(img, s_thresh=(100,255), v_thresh=(50,255)):
    #Use color transforms to create a thresholded binary image
    
    color=np.zeros_like(img[:,:,0])
    hls=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
   
    s_channel = hls[:,:,2]
    v_channel = hsv[:,:,2]
    s_binary = np.zeros_like(s_channel)
    v_binary = np.zeros_like(v_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    v_binary[(v_channel >= v_thresh[0]) & (v_channel <= v_thresh[1])] = 1
    
    color[(s_binary==1)&(v_binary==1)]=255
    return color

In [9]:
def combined_thresh(img,s_thresh=(100, 255),v_thresh=(50,255),sx_thresh=(15, 255),sy_thresh=(25, 255)): 
    #Combine the thresholded binary images using gradients and color transforms
    img = cv2.undistort(img, mtx, dist, None, mtx)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    grad=abs_sobel_thresh(img, sx_thresh, sy_thresh)
    color=color_threshold(img, s_thresh, v_thresh)
    
    combined_binary = np.zeros_like(gray)
    combined_binary[(grad == 255) | (color == 255)] = 255
    return combined_binary

In [10]:
def region_of_interest(img, vertices):
    #Only keeps the region of the image defined by the polygon
    #formed from `vertices`. The rest of the image is set to black.
    
    #actually doesn't have much effect because warp function will also limite the areas of image later
    
    mask=np.zeros_like(img)
    ignore_mask_color=255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image=cv2.bitwise_and(img, mask)
    return masked_image

In [11]:
def warp(img):
    #performe a perspective transform and store examples of a transformed image.
    
    img_size=(img.shape[1],img.shape[0])
    bot_width=0.76
    mid_width=0.09
    height_pct=0.63
    bottom_trim=0.94
    
    src=np.float32([[img.shape[1]*(0.5-mid_width/2),img.shape[0]*height_pct],
                    [img.shape[1]*(0.5+mid_width/2),img.shape[0]*height_pct],
                    [img.shape[1]*(0.5+bot_width/2),img.shape[0]*bottom_trim],
                    [img.shape[1]*(0.5-bot_width/2),img.shape[0]*bottom_trim]])
    offset=img_size[0]*.25
    dst=np.float32([[offset,0],
                    [img_size[0]-offset,0],
                    [img_size[0]-offset,img_size[1]],
                    [offset,img_size[1]]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv=cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M, Minv

In [12]:
def smooth(binary_warped, left_lane_inds, right_lane_inds, nonzerox, nonzeroy): 
    #using the current and last frame to improve the stability of lanes
    #provide coordinations of pixels to calculate the radius
    
    now_leftx = nonzerox[left_lane_inds]
    now_lefty = nonzeroy[left_lane_inds] 
    now_rightx = nonzerox[right_lane_inds]
    now_righty = nonzeroy[right_lane_inds]
    
    if frame==0:
        last_leftx=[]
        last_lefty=[]
        last_rightx=[]
        last_righty=[]
    
    leftx = np.concatenate((now_leftx, last_leftx))
    lefty = np.concatenate((now_lefty, last_lefty))
    rightx= np.concatenate((now_rightx, last_rightx))
    righty= np.concatenate((now_righty, last_righty))
    
    global last_leftx, last_lefty, last_rightx, last_righty
    last_leftx  = now_leftx
    last_lefty  = now_lefty
    last_rightx = now_rightx
    last_righty = now_righty
    
    global left_fit, right_fit
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]   
    
    plt.clf()
    out_img=np.dstack((binary_warped, binary_warped, binary_warped))*255
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    plt.imshow(out_img)
    plt.plot(left_fitx,ploty,color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.savefig("../output_images/lane_lines.jpg")
    plt.clf()
    return ploty,left_fitx,right_fitx

<ipython-input-12-8d75f72059d6>:21: SyntaxWarning: name 'last_leftx' is assigned to before global declaration
  global last_leftx, last_lefty, last_rightx, last_righty
<ipython-input-12-8d75f72059d6>:21: SyntaxWarning: name 'last_lefty' is assigned to before global declaration
  global last_leftx, last_lefty, last_rightx, last_righty
<ipython-input-12-8d75f72059d6>:21: SyntaxWarning: name 'last_rightx' is assigned to before global declaration
  global last_leftx, last_lefty, last_rightx, last_righty
<ipython-input-12-8d75f72059d6>:21: SyntaxWarning: name 'last_righty' is assigned to before global declaration
  global last_leftx, last_lefty, last_rightx, last_righty


In [13]:
def first_frame(binary_warped): 
    
    #determine the very start points of two lanes at the bottom of picture
    histogram = np.sum(binary_warped[3*binary_warped.shape[0]//4:,:], axis=0)
    midpoint = np.int(histogram.shape[0]//2)
    window_width=25
    window=np.ones(window_width)
    leftx_base=np.argmax(np.convolve(window,histogram[:midpoint]))-window_width/2
    rightx_base=np.argmax(np.convolve(window,histogram[midpoint:]))-window_width/2+int(binary_warped.shape[1]/2)

    #set the coefficients for sliding window
    nwindows = 9
    window_height = np.int(binary_warped.shape[0]//nwindows)
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base
    margin = 100
    minpix = 50
    left_lane_inds = []
    right_lane_inds = []

    #using sliding windows to find the lane pixels for first frame
    for window in range(nwindows):
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
 
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    
    global last_left, last_right
    last_left = left_lane_inds
    last_right = right_lane_inds
    ploty, left_fitx, right_fitx = smooth(binary_warped, left_lane_inds, right_lane_inds, nonzerox, nonzeroy)
    
    
    return ploty, left_fitx, right_fitx

In [14]:
def not_first_frame(binary_warped):
    #find lane pixels based on lanes in last frame
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                      left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                      left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                      right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                      right_fit[1]*nonzeroy + right_fit[2] + margin)))  
    ploty, left_fitx, right_fitx = smooth(binary_warped, left_lane_inds, right_lane_inds, nonzerox,nonzeroy)    
        
    return ploty, left_fitx, right_fitx

In [15]:
def calculate_radius(warped, ploty, left_fitx, right_fitx):  
    #calculate the radius and 
    
    ym_per_pix=20/720
    xm_per_pix=3/700
    y_eval=np.max(ploty)

    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
        
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    camera_center=(left_fitx[-1]+right_fitx[-1])/2
    center_diff=(camera_center-warped.shape[1]/2)*xm_per_pix
    side_pos='left'
    if center_diff<=0:
        side_pos='right'
    return left_curverad, right_curverad, center_diff,side_pos

In [16]:
def visualization(warped, undist, ploty, left_fitx, right_fitx, Minv,left_curverad,center_diff,side_pos):   
    #visulize the area between detected lanes
    
    warp_zero = np.zeros_like(warped).astype(np.uint8) 
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
  
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    newwarp = cv2.warpPerspective(color_warp, Minv,(warped.shape[1], warped.shape[0]))   
    result_BGR = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)  
    cv2.putText(result_BGR, 'Radius of Curvature ='+str(round(left_curverad, 3))+'(m)',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result_BGR, 'Vehicle is '+str(abs(round(center_diff,3)))+'m '+side_pos+' of center',(50,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255), 2)
    
    return result_BGR

In [17]:
def pipeline(image): 
    
    undist=cv2.undistort(image, mtx, dist, None, mtx)
    wbimg=combined_thresh(undist,s_thresh=(100, 255),v_thresh=(100,255),sx_thresh=(25, 255),sy_thresh=(25, 255))
    #write_name="../output_images/thresholded_binary-"+str(idx)
    #cv2.imwrite(write_name, wbimg)
    
    vertices=np.array([[(160,680),(600,430),(680,430),(1120,680)]],dtype=np.int32)
    masked_wbimg= region_of_interest(wbimg, vertices)
    
    global M, Minv
    warped, M, Minv = warp(masked_wbimg)
    #write_name="../output_images/wraped-"+str(idx)
    #cv2.imwrite(write_name,warped)
    
    if frame==0:     
        ploty,left_fitx,right_fitx=first_frame(warped)
    else: 
        ploty,left_fitx,right_fitx=not_first_frame(warped)
    global frame
    frame+=1
    
    left_radius, right_radius,center_diff,side_pos=calculate_radius(warped,ploty, left_fitx, right_fitx)
    result=visualization(warped, undist ,ploty, left_fitx, right_fitx, Minv,left_radius,center_diff, side_pos)
    #write_name="../output_images/result_"+str(idx)
    #cv2.imwrite(write_name,result)
    
    return result

<ipython-input-17-0ec9deb14770>:20: SyntaxWarning: name 'frame' is used prior to global declaration
  global frame


In [18]:
import os
images=os.listdir("../test_images/")

for img in images:
    path='../test_images/'+img
    image=cv2.imread(path)
    global frame
    frame=0
    pipeline(image)   

<Figure size 432x288 with 0 Axes>

In [19]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [20]:
global frame
frame=0
def process_image(image):
    img=pipeline(image)
    return img

In [21]:
white_output = '../project_result.mp4'
clip1 = VideoFileClip('../project_video.mp4')
white_clip=clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video ../project_result.mp4
[MoviePy] Writing video ../project_result.mp4


100%|█████████▉| 1260/1261 [18:34<00:00,  1.13it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../project_result.mp4 

CPU times: user 10min 50s, sys: 1min 36s, total: 12min 27s
Wall time: 18min 35s


<Figure size 432x288 with 0 Axes>

In [22]:
HTML("""<video width="1280" height="720" controls>
        <source src={0}>
        </video>
        """.format(white_output))